
# Evalutating different fitness functions for EvoGFuzz

In our project we implement the three given fitness functions with a ```naive```, an ```improved``` and a ```sophisticated``` approach, that were given in the **EvoGFuzz** paper. We then came up with a new approach that uses and aims to improve the ```sophisticated``` approach. We call it the ```ratioed sophisticated``` approach. In this notebook we evaluate each of the approach.


We use the same example as **EvoGFuzz** and therefore need to define our calculator, its oracle and the grammar.

In [1]:
import math

def calculator(inp: str) -> float:
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [2]:
# Make sure you use the OracleResult from the debugging_framework library
from debugging_framework.input.oracle import OracleResult

def oracle(inp: str):
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING
    
    return OracleResult.PASSING

In [3]:
from debugging_framework.types import Grammar
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALCGRAMMAR)

For the new fitness functions we also need to define some helper functions, that we use later on. 

We start by defining a function ```count_expansions``` for the ```improved``` approach, that returns the expansions of a tree, when given the first children of the root of the derivation tree.

We then define the ```calculate_sophisticated_score_structure``` function, that calculated the score structure which considers more complex expansions that are more nested. We sum up the degree exponantiated with the height of the individual nodes.

In [4]:
def count_expansions(children):
    if children == []:
        return 0

    counter = 1

    for child in children:
        _, next_children = child
        counter += count_expansions(next_children)    
    
    return counter

In [5]:
def calculate_sophisticated_score_structure(children, height):
    score = 0
    
    for child in children:
        node, next_children = child        
        score += len(next_children)**height
        score += calculate_sophisticated_score_structure(next_children, height+1)

    return score

In [6]:
def calculate_height_and_degreeSum(children, height):
    max_height = height
    score = 0
    
    for child in children:
        node, next_children = child
        score += len(next_children)**height
        next_score, next_height = calculate_height_and_degreeSum(next_children, height+1)
        score += next_score
        if next_height > max_height:
            max_height = next_height
            
    return score, max_height

Fitness functions:
TODO

In [7]:
from evogfuzz.input import Input

In [8]:
def naive_fitness_function(test_input: Input) -> int:
    score_structure = len(str(test_input))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [9]:
def improved_fitness_function(test_input: Input) -> float:
    _, children = test_input.tree
    number_expansions = count_expansions(children)
    lam = 100
    score_structure = (number_expansions**2)/(lam * len(str(test_input)))
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [10]:
def sophisticated_fitness_function(test_input: Input) -> float:
    score_structure = calculate_sophisticated_score_structure([test_input.tree],0)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 2**100
    else:
        score_feedback = 0
    return score_feedback + score_structure

In [11]:
def ratio_sophisticated_fitness_function(test_input: Input) -> float:
    lam = 2**50
    degreeSums, height = calculate_height_and_degreeSum([test_input.tree],0)
    score_structure = degreeSums/(lam * height)
    if test_input.oracle == OracleResult.FAILING:
        score_feedback = 100
    else:
        score_feedback = 0
    return score_feedback + score_structure

Finally we can define EvoGFuzz instances with all different fitness functions. For comparison, we start with defining an instance with the standard fitness function and thereafter define one for each of ours.


In [15]:
from evogfuzz.evogfuzz_class import EvoGFuzz

def eval_fitness(eval_iterations,initial_inputs,iterations):
    """
    Evaluate the differnet fitness functions.
    :param eval_iterations: The number of iterations we use to calculate the found exception inputs.
    :param initial_inputs: The input from which EvoGFuzz starts to train.
    :param itarations: The number of iterations EvoGFuzz trains.
    :return: The total number of found exception inputs.
    """   
    
    len_stand = 0
    len_naive = 0
    len_impr = 0
    len_soph = 0
    len_ratio_soph = 0
    
    for i in range(eval_iterations):
        epp_stand = EvoGFuzz(
            grammar=CALCGRAMMAR,
            oracle=oracle,
            inputs=initial_inputs,
            iterations=iterations
        )
        
        epp_naive = EvoGFuzz(
            grammar=CALCGRAMMAR,
            oracle=oracle,
            inputs=initial_inputs,
            fitness_function=naive_fitness_function,
            iterations=iterations
        )
        
        epp_impr = EvoGFuzz(
            grammar=CALCGRAMMAR,
            oracle=oracle,
            inputs=initial_inputs,
            fitness_function=improved_fitness_function,
            iterations=iterations
        )
        
        epp_soph = EvoGFuzz(
            grammar=CALCGRAMMAR,
            oracle=oracle,
            inputs=initial_inputs,
            fitness_function=sophisticated_fitness_function,
            iterations=iterations
        )

        epp_ratio_soph = EvoGFuzz(
            grammar=CALCGRAMMAR,
            oracle=oracle,
            inputs=initial_inputs,
            fitness_function=ratio_sophisticated_fitness_function,
            iterations=iterations
        )
        
        found_exc_inp_stand = epp_stand.fuzz()
        found_exc_inp_naive = epp_naive.fuzz()
        found_exc_inp_impr = epp_impr.fuzz()
        found_exc_inp_soph = epp_soph.fuzz()
        found_exc_inp_ratio_soph = epp_ratio_soph.fuzz()
    
        len_stand += len(found_exc_inp_stand)
        len_naive += len(found_exc_inp_naive)
        len_impr += len(found_exc_inp_impr)
        len_soph += len(found_exc_inp_soph)
        len_ratio_soph += len(found_exc_inp_ratio_soph)

        return len_stand, len_naive, len_impr, len_soph, len_ratio_soph


In [ ]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

len_stand, len_naive, len_impr, len_soph, len_ratio_soph = eval_fitness(eval_iterations=100, initial_inputs=initial_inputs, iterations=10)

print("standard:", len_stand)
print("naive:", len_naive)
print("improved:", len_impr)
print("sophisticated:", len_soph)
print("ratio_sophisticated:", len_ratio_soph)